In [ ]:
train_data = utils.image_dataset_From directory(
    "/app/data/lego-brick-images/dataset/",
    labels = None,
    color_mode = "grayscale",
    image_size = (64, 64),
    batch_size = 128,
    shuffle = True,
    seed = 42,
    interpolation = "bilinear",
)

In [ ]:
def preprocess(img):
    img = (tf.cast(img, "float32") - 127.5) / 127.5
    return img

train = train_data.map(lambda x: preprocess(x))

In [ ]:
discriminator_input = layers.Input(shape = (64, 64, 1))
x = layers.Conv2D(64, kernel_size = 4, strides = 2, padding = "same", use_bias = False)()
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(
    128, kernel_size = 4, strides = 2, padding = "same", use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(
    256, kernel_size = 4, strides = 2, padding = "same", use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(
    512, kernel_size = 4, strides = 2, padding = "same", use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(
    1, kernel_size = 4, strides = 1, padding = "valid", use_bias = False, activation = 'sigmoid'
)(x)
discriminator_output = layers.Flatten()(x)

discriminator = models.Model(discriminator_input, discriminator_output)

In [ ]:
generator_input = layers.Input(shape = (100,))
x = layers.Reshape((1, 1, 100))(generator_input)
x = layers.Conv2DTranspose(
    512, kernel_size = 4, strides = 1, padding = "valid", use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Conv2DTranspose(
    256, kernel_size = 4, strides = 1, padding = "valid", use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Conv2DTranspose(
    128, kernel_size = 4, strides = 1, padding = "valid", use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Conv2DTranspose(
    64, kernel_size = 4, strides = 1, padding = "valid", use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
generator_output = layers.Conv2DTranspose(
    1,
    kernel_size = 4,
    strides = 2,
    padding = "same",
    use_bias = False,
    activation = 'tanh'
)(x)
generator = models.Model(generator_input, generator_output)

In [ ]:
class DCGAN(models.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(DCGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer):
        super(DCGAN, self).compile()
        self.loss_fn = losses.binaryCrossentropy()
        self.d_optimizer = d_optimizer
        self.g_optimizer = d_optimizer
        self.d_loss_metric = metrics.Mean(name = "d_loss")
        self.g_loss_metric = metrics.Mean(name = "g_loss")
    
    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]
    
    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        random_latent_Vectors = tf.random.normal(
            shape = (batch_size, self.latent_dim)
        )

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = self.generator(
                random_latent_vectors, training = True
            )
            real_predictions = self.discriminator(
                real_images, training = True
            )
            fake_predictions = self.discriminator(
                generated_images, trainig = True
            )

            real_labels = tf.ones_like(real_predictions)
            real_noisy_labels = real_labels + 0.1 * tf.random.uniform(
                tf.shape(real_predictions)
            )

            fake_labels = tf.zeros_like(fake_predictions)
            fake_noisy_labels = fake_labels - 0.1 * tf.random.uniform(
                tf.shape(fake_predictions)
            )

            d_real_loss = self.loss_fn(real_noisy_labels, real_predictions)
            d_fake_loss = self.loss_fn(fake_noisy_labels, fake_predictions)
            d_loss = (d_real_loss + d_fake_loss) / 2.0

            g_loss = self.loss_fn(real_labels, fake_predictions)

        gradients_of_discriminator = disc_tape.gradient(
            d_loss, self.discriminator.trainable_variables
        )
        gradients_of_generator = gen_tape.gradient(
            g_loss, self.generator.trainable_variables
        )

        self.d_optimizer.apply_gradients(
            zip(gradients_of_discriminator, discriminator.trainable_variables)
        )
        self.g_optimizer.apply_gradients(
            zip(gradients_of_generator, generator. trainable_variables)
        )

        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)

        return {m.name: m.result for m in self.metrics}
    
    dcgan = DCGAN(
        discriminator = discriminator, generator = generator, latent_dim = 100
    )

    dcgan.compile(
        d_optimizer = optimizers.Adam(
            learning_rate = 0.0002, beta_1 = 0.5, beta_2 = 0.999
        ),
        g_optimizer = optimizers.Adam(
            learning_rate = 0.0002, beta_1 = 0.5, beta_2 = 0.999
        ),
    )

    dcgan.fit(train, epochs = 300)

In [ ]:
def gradient_penalty(self, batch_size, real_images, fake_images):
    alpha = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
    diff = fake_images - real_images
    interpolated = real_images + alpha * diff

    with tf.GradientTape() as gp_tape:
        gp_tape.watch(interpolated)
        pred = self.critic(interpolated, training = True)
    
    grads = gp_tape.gradient(pred, [interpolated])[0]
    norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis = [1, 2, 3]))
    gp = tf.reduce_mean((norm - 1.0) ** 2)
    return gp

In [ ]:
def train_step(self, real_images):
    batch_size = tf.shape(real_images)[0]

    for i in range(3):
        random_latent_vectors = tf.random.normal(
            shape = (batch_size, self.latent_dim)
        )

        with tf.Gradienttape() as tape:
            fake_images = self.generator(
                random_latent_vectors, training = True
            )
            fake_predictions = self.critic(fake_images, training = True)
            real_predictions = self.critic(real_images, training = True)

            c_wass_loss = tf.reduce_mean(fake_predictions) - tf.reduce_mean(
                real_predictions
            )
            c_gp = self.gradient_penalty(
                batch_size, real_images, fake_images
            )
            c_loss = c_wass_loss + c_gp * self.gp_weight

        c_gradient = tape.gradient(c_loss, self.critic.trainable_variables)

        self.c_optimizer.apply_gradients(
            zip(c_gradient, self.critic.trainable_variables)
        )

    random_latent_vectors = tf.random.normal(
        shape = (batch_size, self.latent_dim)
    )
    with tf.GradientTape() as tape:
        fake_images = self.generator(random_latent_vectors, training = True)
        fake_predictions = self.critic(fake_images, training = True)
        g_loss = - tf.reduce_mean(fake_predictions)
    
    gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
    self.g_optimizer.apply_gradients(
        zip(gen_gradient, self.generator.trainable_variables)
    )

    self.c_loss_metric.update_state(c_loss)
    self.c_wass_loss_metric.update_state(c_wass_loss)
    self.c_gp_metric.update_state(c_gp)
    self.c_gp_metric.update_state(c_loss)

    return {m.name: m.result() for m in self.metrics}

In [ ]:
def train_step(self, data):
    real_images, one_hot_labels = data

    image_one_hot_labels = one_hot_labels[:, None, None, :]
    image_one_hot_labels = tf.repeat(
        image_one_hot_labels, repeats = 64, axis = 1
    )
    image_one_hot_labels = tf.repeat(
        image_one_hot_labels, repeats = 64, axis = 2
    )

    batch_size = tf.shape(real_images)[0]

    for i in range(self.critic_steps):
        random_latent_vectors = tf.random.normal(
            shape = (batch_size, self.latent_dim)
        )

        with tf.Gradienttape() as tape:
            fake_images = self.generator(
                [random_latent_vectors, one_hot_labels], training = True
            )
            fake_predictions = self.critic([fake_images, image_one_hot_labels], training = True)
            real_predictions = self.critic([real_images, image_one_hot_labels], training = True)

            c_wass_loss = tf.reduce_mean(fake_predictions) - tf.reduce_mean(
                real_predictions
            )
            c_gp = self.gradient_penalty(
                batch_size, real_images, fake_images, image_one_hot_labels
            )
            c_loss = c_wass_loss + c_gp * self.gp_weight

        c_gradient = tape.gradient(c_loss, self.critic.trainable_variables)
        self.c_optimizer.apply_gradients(
            zip(c_gradient, self.critic.trainable_variables)
        )
    
    random_latent_vectors = tf.random.normal(
            shape = (batch_size, self.latent_dim)
    )

    with tf.Gradienttape() as tape:
        fake_images = self.generator(
            [random_latent_vectors, one_hot_labels], training = True
        )
        fake_predictions = self.critic([fake_images, image_one_hot_labels], training = True)
        g_loss = -tf.reduce_mean(fake_predictions)
    
    gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
    self.c_optimizer.apply_gradients(
        zip(c_gradient, self.critic.trainable_variables)
    )